This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [24]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [16]:
import pandas as pd
import numpy as np

N = 50000
data = 'data/data_000637.txt'
df = pd.read_csv(data,nrows=N) # To read and store the data from the file and create a dataframe

df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
49995,1,1,2,3869200664,1103,4
49996,1,1,4,3869200664,1105,14
49997,1,0,59,3869200664,1191,17
49998,1,0,32,3869200664,1191,1


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [8]:
x = max(df['BX_COUNTER'])+1 #Here max value is added by 1 since 1min = 60sec and after 59th sec timer resets to 0
                            #Therefore 1 is added

print("The value of x=",x)

The value of x= 3564


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [17]:
abs_time = df['TDC_MEAS']*25/30 + df['BX_COUNTER']*25 + df['ORBIT_CNT']*x*25
duration_sec = (abs_time.max() - abs_time.min())*1e-9 
print('Duration in seconds: ', duration_sec)

Duration in seconds:  0.0442531175


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [18]:
t_0 = df.at[0,'ORBIT_CNT']*x*25 + df.at[0,'BX_COUNTER']*25 + df.at[0,'TDC_MEAS']*(25/30)

# New column : ABS_T in ns
df['ABS_T in ns'] = df['ORBIT_CNT']*x*25 + df['BX_COUNTER']*25 + df['TDC_MEAS']*(25/30) - t_0
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_T in ns
0,1,0,123,3869200167,2374,26,0.000000e+00
1,1,0,124,3869200167,2374,27,8.125000e-01
2,1,0,63,3869200167,2553,28,4.476625e+03
3,1,0,64,3869200167,2558,19,4.594125e+03
4,1,0,64,3869200167,2760,25,9.649125e+03
...,...,...,...,...,...,...,...
49995,1,1,2,3869200664,1103,4,4.425091e+07
49996,1,1,4,3869200664,1105,14,4.425096e+07
49997,1,0,59,3869200664,1191,17,4.425312e+07
49998,1,0,32,3869200664,1191,1,4.425310e+07


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [19]:
df.groupby('TDC_CHANNEL').size().sort_values(ascending=False)[:3]

TDC_CHANNEL
139    4175
64     2523
63     2472
dtype: int64

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [23]:
non_empty = df['ORBIT_CNT'].unique().shape[0] 
print('Non-empty orbits: ', non_empty)

tdc139 = df[df['TDC_CHANNEL'] == 139]
uni_orbit = tdc139['ORBIT_CNT'].unique().shape[0]
print('Unique orbits from TDC_CHANNEL=139: ', uni_orbit)

Non-empty orbits:  498
Unique orbits from TDC_CHANNEL=139:  498


7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel